In [1]:
from piper import piper
from piper.verbs import *

piper version 0.0.7, last run: Friday, 19 February 2021 11:58:42


In [2]:
year = 2020

url = f"https://en.wikipedia.org/wiki/List_of_British_films_of_{year}"
logger.info(url)
tables = pd.read_html(url, header=0)

columns = ['month', 'day', 'title', 'cast crew', 'genre', 'ref']

https://en.wikipedia.org/wiki/List_of_British_films_of_2020


In [3]:
%%piper 
df <- pd.concat(tables[1:5]) 
>> set_columns(columns) 
>> clean_columns() 
>> where("~day.isna()") 
>> drop(columns='ref') 
>> assign(month=lambda x: x.month.str.title()) 
>> assign(day=lambda x: x.day.astype(int)) 
>> assign(release_date=lambda x: pd.to_datetime(f'{year} ' + x.month + x.day.astype(str))) 
>> move_column('release_date', loc=0) 
>> drop(columns=['month', 'day']) 
>> assign(director=lambda x: x.cast_crew.str.extract(r'Director[s]?:\s([\s,\w]*)\n?')) 
>> assign(director=lambda x: x.director.str.strip().str.replace('Cast', '')) 
>> assign(director=lambda x: x.director.str.strip()) 
>> assign(cast=lambda x: x.cast_crew.str.strip().str.extract('Cast:(.*)')) 
>> assign(cast=lambda x: x.cast.str.strip().str.split(pat=', ')) 
>> explode('cast') 
>> drop(columns='cast_crew')
>> pd.DataFrame.reset_index(drop=True)

In [4]:
%%piper 
count(df, 'cast') >>
where("cast.isna()") >>
head(5)

0 rows, 2 columns


,cast,n


In [5]:
df = df[~df.cast.isna()]

In [6]:
df.query("cast.str.contains('Ferguson')")

,title,genre,director,cast


In [7]:
# %%piper
# df >> head(5)

In [8]:
head(df)

210 rows, 4 columns


,title,genre,director,cast
0,Come Away,GEM Entertainment Based on Alice's Adventures ...,Brenda Chapman,Angelina Jolie
1,Come Away,GEM Entertainment Based on Alice's Adventures ...,Brenda Chapman,David Oyelowo
2,Come Away,GEM Entertainment Based on Alice's Adventures ...,Brenda Chapman,Gugu Mbatha-Raw
3,Come Away,GEM Entertainment Based on Alice's Adventures ...,Brenda Chapman,Michael Caine
